## **0. Change working directory**

In [60]:
ROOT = "/Users/macbookairm4chip/Desktop/DAM202/Practical2" # Your Working Directory
import os
os.chdir(ROOT)

- Set up working directory for the practical.

In [61]:
#import os
os.listdir()

['optimized_alice_word2vec.model',
 'bin',
 'Untitled.ipynb',
 'include',
 'etc',
 'pyvenv.cfg',
 'my_word2vec_model.model',
 'cbow_word2vec_model.model',
 'lib',
 'text.txt',
 '.ipynb_checkpoints',
 'share']

- Lists all the files and folders in the directory 


## **1. Introduction**

### **1.1 Need for Data Feed**

While pretrained models like Google News Word2Vec are powerful, training our own model offers several advantages:

* **Domain Specificity**: Captures terminology and relationships specific to our field (medical, legal, technical)
* **Custom Vocabulary**: Includes words and phrases unique to your dataset
* **Control**: Full control over training parameters and data quality
* **Privacy**: No need to rely on external models for sensitive data
* **Learning**: Deep understanding of how Word2Vec actually works

### **The Neural Network Architecture**

Word2Vec uses a simple neural network with three layers:

* **Input Layer**: One-hot encoded word vectors
* **Hidden Layer**: Dense representation (the embeddings we want)
* **Output Layer**: Probability distribution over vocabulary

### **1.2 CBOW vs Skip-gram Training**

#### **CBOW (Continuous Bag of Words):**
* **Input**: Context words → **Output**: Center word
* **Example**: ["the", "cat", "on", "mat"] → "sat"
* **Advantages**: 
  - Faster training
  - Better for frequent words
  - Good for syntactic relationships

#### **Skip-gram:**
* **Input**: Center word → **Output**: Context words
* **Example**: "sat" → ["the", "cat", "on", "mat"]
* **Advantages**:
  - Better for rare words
  - Excellent for semantic relationships
  - Slower training but higher quality

## **2. Training Objectives**

The model learns by:
* **Maximizing** probability of actual word pairs that appear together
* **Minimizing** probability of random word pairs (negative sampling)
* **Adjusting** word vectors to achieve these objectives

### **2.1 Key Training Concepts**

#### **Context Window**
Number of words around target word to consider:
* **Small window (2-3)**: Captures syntactic relationships
* **Large window (5-10)**: Captures semantic/topical relationships

#### **Negative Sampling**
Instead of computing probabilities for entire vocabulary, sample a few "negative" examples:
* Dramatically speeds up training
* 5-20 negative samples typically used

#### **Hierarchical Softmax**
Alternative to negative sampling using binary tree structure:
* Better for rare words
* More memory efficient for large vocabularies



### **3.1 Data Collection and Preparation**


In [62]:
with open('text.txt', 'r', encoding='utf-8') as f: # Remember your data set path should be specified if not in same working directory
    texts = f.readlines()

What it does:

- Opens the Alice in Wonderland text file
- Reads every line from the file
- Puts all those lines into a list called texts
- Each line becomes one item in the list

In [63]:
texts[:10]

["                ALICE'S ADVENTURES IN WONDERLAND\n",
 '\n',
 '                          Lewis Carroll\n',
 '\n',
 '               THE MILLENNIUM FULCRUM EDITION 3.0\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '                            CHAPTER I\n']

### **3.2 Data Quality Assessment**


In [64]:
def assess_data_quality(texts):
    """Analyze text data quality for Word2Vec training"""

    stats = {
        'total_documents': len(texts),
        'total_words': 0,
        'unique_words': set(),
        'sentence_lengths': [],
        'word_frequencies': {}
    }

    for text in texts:
        words = text.lower().split()
        stats['total_words'] += len(words)
        stats['sentence_lengths'].append(len(words))
        stats['unique_words'].update(words)

        for word in words:
            stats['word_frequencies'][word] = stats['word_frequencies'].get(word, 0) + 1

    stats['vocabulary_size'] = len(stats['unique_words'])
    stats['avg_sentence_length'] = sum(stats['sentence_lengths']) / len(stats['sentence_lengths'])

    # Find most common words
    sorted_words = sorted(stats['word_frequencies'].items(), key=lambda x: x[1], reverse=True)
    stats['top_words'] = sorted_words[:20]

    # Quality indicators
    stats['quality_score'] = {
        'vocabulary_diversity': stats['vocabulary_size'] / stats['total_words'],
        'avg_word_frequency': stats['total_words'] / stats['vocabulary_size'],
        'rare_words_ratio': sum(1 for count in stats['word_frequencies'].values() if count == 1) / stats['vocabulary_size']
    }

    return stats

# Example usage
quality_report = assess_data_quality(texts)
print(f"Total documents: {quality_report['total_documents']:,}")
print(f"Vocabulary size: {quality_report['vocabulary_size']:,}")
print(f"Unique Words: {quality_report['unique_words']}")
print(f"Average sentence length: {quality_report['avg_sentence_length']:.1f}")
print(f"Vocabulary diversity: {quality_report['quality_score']['vocabulary_diversity']:.4f}")

Total documents: 3,598
Vocabulary size: 4,950
Unique Words: {'hedgehogs;', "extras?'", 'turkey,', 'feeble,', "shouldn't", "i--'", 'labelled', 'sneezed', '`because', 'king', 'soothing', "is--'", 'their', 'ring,', "nonsense!'", "answer?'", '"much', "sir,'", '(with', 'character,', 'bank,', 'rapidly;', 'turtle:', 'accidentally', 'morning,', "stay!'", "`shan't,'", "toes?'", 'herself,', '`this', 'parchment', "`she'd", "stuff,'", "is,'", 'peering', 'me!', '`for', 'queer-', 'form', "done.'", 'melancholy', 'thinking', "hot-tempered,'", "trial.'", 'like,', 'conversation.', 'shaped', 'writhing,', 'a-piece', "pleases!'", 'sea,', "mind,'", 'course--', 'lives.', 'lest', 'promising,', 'escape', 'tasted', 'bird', 'hedgehog', 'cheeks,', "we've", 'beasts,', 'sharply.', 'till', 'waited', 'how', 'join', 'officers', 'mushroom', "asleep,'", 'farm-yard--while', 'pleaded', 'pulling', "again!'", 'invited', 'scream', "caucus-race.'", 'round,', 'yesterday,', 'able!', 'loudly', 'opened', 'shakespeare,', 'answered

This code **analyzes the text dataset and gives a detailed quality report.**

**What it does:**

- Counts total documents, words, and unique vocabulary

- Calculates average sentence length and word frequencies

- Finds the 20 most common words

- Measures vocabulary diversity (how varied the words are)

- Identifies rare words that appear only once

- Generates quality scores to assess if the dataset is good for Word2Vec training

**Result:** We get statistics about Alice in Wonderland text - like how many unique words it has, average sentence length, and whether the vocabulary is rich enough for effective Word2Vec training.

### **3.3 Text Preprocessing Pipeline**


In [65]:
%pip install nltk

#Import Packages
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
import nltk

Note: you may need to restart the kernel to use updated packages.


In [66]:
# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/macbookairm4chip/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/macbookairm4chip/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/macbookairm4chip/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/macbookairm4chip/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/macbookairm4chip/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [67]:
class AdvancedTextPreprocessor:
    """Comprehensive text preprocessing for Word2Vec training"""

    def __init__(self,
                 lowercase=True,
                 remove_punctuation=True,
                 remove_numbers=False,
                 remove_stopwords=False,
                 min_word_length=2,
                 max_word_length=50,
                 lemmatize=False,
                 remove_urls=True,
                 remove_emails=True,
                 keep_sentences=True):

        self.lowercase = lowercase
        self.remove_punctuation = remove_punctuation
        self.remove_numbers = remove_numbers
        self.remove_stopwords = remove_stopwords
        self.min_word_length = min_word_length
        self.max_word_length = max_word_length
        self.lemmatize = lemmatize
        self.remove_urls = remove_urls
        self.remove_emails = remove_emails
        self.keep_sentences = keep_sentences

        if remove_stopwords:
            self.stop_words = set(stopwords.words('english'))

        if lemmatize:
            self.lemmatizer = WordNetLemmatizer()

    def clean_text(self, text):
        """Clean individual text string"""

        # Remove URLs
        if self.remove_urls:
            text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

        # Remove email addresses
        if self.remove_emails:
            text = re.sub(r'\S+@\S+', '', text)

        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text).strip()

        #Combined
         #(r'https?://\S+|www\.\S+|<.*?>|\S+@\S+\.\S+|@\w+|#\w+|[^A-Za-z0-9\s])

        return text

    def tokenize_text(self, text):
        """Tokenize text into sentences or words"""

        if self.keep_sentences:
            # Tokenize into sentences first
            sentences = sent_tokenize(text)
            processed_sentences = []

            for sentence in sentences:
                words = self.process_sentence(sentence)
                if len(words) >= 3:  # Keep sentences with at least 3 words
                    processed_sentences.append(words)

            return processed_sentences
        else:
            # Return single list of words
            return self.process_sentence(text)

    def process_sentence(self, sentence):
        """Process individual sentence"""

        # Lowercase
        if self.lowercase:
            sentence = sentence.lower()

        # Tokenize into words
        words = word_tokenize(sentence)

        processed_words = []
        for word in words:

            # Remove punctuation
            if self.remove_punctuation:
                word = word.translate(str.maketrans('', '', string.punctuation))

            # Skip if empty after punctuation removal
            if not word:
                continue

            # Remove numbers
            if self.remove_numbers and word.isdigit():
                continue

            # Check word length
            if len(word) < self.min_word_length or len(word) > self.max_word_length:
                continue

            # Remove stopwords
            if self.remove_stopwords and word in self.stop_words:
                continue

            # Lemmatize
            if self.lemmatize:
                word = self.lemmatizer.lemmatize(word)

            processed_words.append(word)

        return processed_words

    def preprocess_corpus(self, texts):
        """Preprocess entire corpus"""

        all_sentences = []

        for text in texts:
            if not isinstance(text, str):
                continue

            # Clean text
            cleaned_text = self.clean_text(text)

            # Tokenize and process
            processed = self.tokenize_text(cleaned_text)

            if self.keep_sentences:
                all_sentences.extend(processed)
            else:
                all_sentences.append(processed)

        return all_sentences

**What it does:**

- **Cleans text**: Removes URLs, emails, extra spaces, and punctuation

- **Converts to lowercase**: Makes all words consistent 

- **Tokenizes**: Splits text into sentences and individual words

- **Filters words**: Removes very short/long words, numbers, and stopwords if needed

- **Processes entire dataset**: Takes raw text and converts it into clean, organized sentences

- **Customizable**: turn different cleaning options on/off

**Result:** Transforms messy raw text like "Alice's Adventures in Wonderland!" into clean word lists like ["alice", "adventures", "wonderland"] that Word2Vec can effectively learn from. This preprocessing step is crucial for good Word2Vec performance.

In [68]:
# Example usage
preprocessor = AdvancedTextPreprocessor(
    lowercase=True,
    remove_punctuation = True,
    remove_numbers=True,
    remove_stopwords=False,  # Keep stopwords for Word2Vec
    lemmatize=False,  # Usually not needed for Word2Vec
    keep_sentences=True
)

# Processing corpus
processed_sentences = preprocessor.preprocess_corpus(texts)
print(f"Processed {len(processed_sentences)} sentences")
print(f"Sample sentence: {processed_sentences[0]}")

Processed 2941 sentences
Sample sentence: ['alice', 'adventures', 'in', 'wonderland']



This code **creates a text preprocessor with specific settings and cleans the Alice in Wonderland text**.

**What it does:**
- **Creates preprocessor**: Sets up cleaning rules (lowercase, remove punctuation/numbers, keep stopwords)
- **Processes your dataset**: Runs the preprocessor on the text
- **Converts raw text**: Transforms messy text into clean sentences of word lists
- **Shows results**: Displays how many sentences were processed and shows a sample cleaned sentence



In [69]:
processed_sentences[:3]

[['alice', 'adventures', 'in', 'wonderland'],
 ['the', 'millennium', 'fulcrum', 'edition'],
 ['down', 'the', 'rabbithole']]

### **3.4 Training Parameters**

In [70]:
def recommend_parameters(corpus_size, vocab_size, domain_type, computing_resources):
    """
    Recommend Word2Vec parameters based on corpus characteristics

    Args:
        corpus_size: Number of sentences/documents
        vocab_size: Unique words in vocabulary
        domain_type: 'general', 'technical', 'social_media', 'academic'
        computing_resources: 'limited', 'moderate', 'high'
    """

    recommendations = {}

    # Vector size based on corpus and vocab size
    if corpus_size < 10000:
        recommendations['vector_size'] = 50
    elif corpus_size < 100000:
        recommendations['vector_size'] = 100
    elif corpus_size < 1000000:
        recommendations['vector_size'] = 200
    else:
        recommendations['vector_size'] = 300

    # Window size based on domain
    domain_windows = {
        'general': 5,
        'technical': 3,  # More syntactic focus
        'social_media': 4,
        'academic': 6    # More semantic focus
    }
    recommendations['window'] = domain_windows.get(domain_type, 5)

    # Min count based on corpus size
    if corpus_size < 10000:
        recommendations['min_count'] = 1
    elif corpus_size < 100000:
        recommendations['min_count'] = 2
    elif corpus_size < 1000000:
        recommendations['min_count'] = 5
    else:
        recommendations['min_count'] = 10

    # Algorithm selection
    if domain_type in ['technical', 'academic']:
        recommendations['sg'] = 1  # Skip-gram for rare technical terms
    else:
        recommendations['sg'] = 0  # CBOW for general text

    # Epochs based on corpus size and resources
    if computing_resources == 'limited':
        recommendations['epochs'] = 5
    elif corpus_size < 100000:
        recommendations['epochs'] = 15
    else:
        recommendations['epochs'] = 10

    # Hierarchical softmax vs negative sampling
    if vocab_size > 100000:
        recommendations['hs'] = 1
        recommendations['negative'] = 0
    else:
        recommendations['hs'] = 0
        recommendations['negative'] = 10

    return recommendations


This code **automatically suggests the best Word2Vec training settings** based on the dataset characteristics.

**What it does:**
- **Analyzes the data**: Takes corpus size, vocabulary size, text type, and computing power as input
- **Recommends vector size**: Bigger datasets get higher-dimensional word vectors (50-300 dimensions)
- **Sets window size**: How many surrounding words to consider (3-6 words)
- **Chooses algorithm**: CBOW for general text, Skip-gram for technical/academic text
- **Determines training time**: More epochs for smaller datasets, fewer for larger ones
- **Optimizes performance**: Selects best training method based on vocabulary size

**Result:** Instead of guessing Word2Vec parameters, we get smart recommendations tailored to  dataset - like "use 100-dimension vectors, 5-word window, CBOW algorithm, 15 epochs" for optimal training results.

In [71]:
corpus_size = len(processed_sentences)
print(f"Corpus Size: {corpus_size}")

# Calculate vocabulary size (unique words in vocabulary)
vocab = set(word for sentence in processed_sentences for word in sentence)
vocab_size = len(vocab)
print(f"Vocabulary Size: {vocab_size}")

Corpus Size: 2941
Vocabulary Size: 2519


- Calculates key statistics about processed dataset

In [72]:

params = recommend_parameters(
    corpus_size=corpus_size,
    vocab_size=vocab_size,
    domain_type='academic',
    computing_resources='moderate'
)
print("Recommended parameters:", params)

Recommended parameters: {'vector_size': 50, 'window': 6, 'min_count': 1, 'sg': 1, 'epochs': 15, 'hs': 0, 'negative': 10}


This code **gets personalized Word2Vec training recommendations** for the specific dataset.

**What it does:**
- **Uses data stats**: Takes actual corpus size and vocabulary size
- **Sets domain type**: Specifies 'general' text (since Alice is general literature, not technical)
- **Sets computing power**: Uses 'moderate' resources (standard laptop/desktop)
- **Gets recommendations**: Returns optimal parameter settings tailored to your dataset
- **Shows results**: Displays the recommended settings like vector size, window size, epochs, etc.

**Result:** output like "Recommended parameters: {'vector_size': 100, 'window': 5, 'min_count': 2, 'sg': 0, 'epochs': 15}" - gives the perfect settings to train Word2Vec effectively on the dataset.

### **3.5 Step-by-Step Implementation**

In [73]:
%pip install gensim

Note: you may need to restart the kernel to use updated packages.


**Gensim** is a Python library specifically designed for topic modeling and document similarity analysis. 

Main Purpose:

- Processes large collections of text documents
- Extracts semantic patterns and relationships between words
- Creates mathematical representations of text meaning



In [90]:
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
import time
import multiprocessing
class EpochLogger(CallbackAny2Vec):
    """Callback to log information about training progress"""
    def __init__(self):
        self.epoch = 0
        self.start_time = time.time()
    def on_epoch_begin(self, model):
        print(f"Epoch #{self.epoch} start")
    def on_epoch_end(self, model):
        elapsed = time.time() - self.start_time
        print(f"Epoch #{self.epoch} end - Time elapsed: {elapsed:.2f}s")
        self.epoch += 1
def train_word2vec_model(sentences, save_path=None, **params):
    """
    Train Word2Vec model with given parameters
    Args:
        sentences: List of tokenized sentences
        save_path: Path to save the model
        **params: Word2Vec parameters
    """
    # Set default parameters optimized for Alice in Wonderland dataset
    default_params = {
        'vector_size': 150,
        'window': 8,
        'min_count': 2,
        'workers': multiprocessing.cpu_count() - 1,
        'sg': 0,  # Skip-gram
        'epochs': 50,
        'alpha': 0.025,
        'min_alpha': 0.0001,
        'hs': 0,
        'negative': 15
    }
    # Update with provided parameters
    default_params.update(params)
    print("Training Word2Vec model with parameters:")
    for key, value in default_params.items():
        print(f"  {key}: {value}")
    # Add callback for progress monitoring
    epoch_logger = EpochLogger()
    # Train the model
    print(f"\nTraining on {len(sentences)} sentences...")
    start_time = time.time()
    model = Word2Vec(
        sentences=sentences,
        callbacks=[epoch_logger],
        **default_params
    )
    training_time = time.time() - start_time
    print(f"\nTraining completed in {training_time:.2f} seconds")
    print(f"Vocabulary size: {len(model.wv)} words")
    # Save model if path provided
    if save_path:
        model.save(save_path)
        print(f"Model saved to {save_path}")
    return model

- Sets up the complete Word2Vec training infrastructure with progress monitoring and customizable parameters.

- A complete Word2Vec training system that will show real-time progress as it learns word relationships from the dataset. When we call this function later, it will train the model and give detailed feedback about the training process.

In [91]:
# Example usage optimized for Alice in Wonderland dataset
model = train_word2vec_model(
    sentences=processed_sentences,
    save_path='my_word2vec_model.model',
    vector_size=150,
    window=8,
    min_count=2,
    epochs=50,
    sg=1,
    negative=15,
    compute_loss=True
)

Training Word2Vec model with parameters:
  vector_size: 150
  window: 8
  min_count: 2
  workers: 9
  sg: 1
  epochs: 50
  alpha: 0.025
  min_alpha: 0.0001
  hs: 0
  negative: 15
  compute_loss: True

Training on 2941 sentences...
Epoch #0 start
Epoch #0 end - Time elapsed: 0.07s
Epoch #1 start
Epoch #1 end - Time elapsed: 0.12s
Epoch #2 start
Epoch #2 end - Time elapsed: 0.17s
Epoch #3 start
Epoch #3 end - Time elapsed: 0.22s
Epoch #4 start
Epoch #4 end - Time elapsed: 0.26s
Epoch #5 start
Epoch #5 end - Time elapsed: 0.31s
Epoch #6 start
Epoch #6 end - Time elapsed: 0.36s
Epoch #7 start
Epoch #7 end - Time elapsed: 0.41s
Epoch #8 start
Epoch #8 end - Time elapsed: 0.45s
Epoch #9 start
Epoch #9 end - Time elapsed: 0.50s
Epoch #10 start
Epoch #10 end - Time elapsed: 0.55s
Epoch #11 start
Epoch #11 end - Time elapsed: 0.60s
Epoch #12 start
Epoch #12 end - Time elapsed: 0.65s
Epoch #13 start
Epoch #13 end - Time elapsed: 0.69s
Epoch #14 start
Epoch #14 end - Time elapsed: 0.74s
Epoch #15

- Trains the model on the dataset using the training function defined.

In [92]:
vocab_size = len(model.wv.index_to_key)
print("Vocabulary Size:", vocab_size)

Vocabulary Size: 1396


- Checks how many unique words trained Word2Vec model learned.

In [93]:
all_words = model.wv.index_to_key
print("All Words in Vocabulary:", all_words[:10])

All Words in Vocabulary: ['the', 'and', 'to', 'it', 'she', 'of', 'said', 'you', 'in', 'was']


### **3.6 Model Evaluation and Validation**


In [94]:
%pip install scikit-learn

import numpy as np
from scipy.stats import spearmanr
from sklearn.metrics.pairwise import cosine_similarity

class Word2VecEvaluator:
    """Comprehensive evaluation suite for Word2Vec models"""

    def __init__(self, model):
        self.model = model
        self.wv = model.wv

    def evaluate_word_similarity(self, word_pairs_with_scores):
        """
        Evaluate model on word similarity datasets

        Args:
            word_pairs_with_scores: List of tuples (word1, word2, human_score)

        Returns:
            Spearman correlation with human judgments
        """

        model_similarities = []
        human_similarities = []

        for word1, word2, human_score in word_pairs_with_scores:
            try:
                model_sim = self.wv.similarity(word1, word2)
                model_similarities.append(model_sim)
                human_similarities.append(human_score)
            except KeyError:
                # Skip if words not in vocabulary
                continue

        if len(model_similarities) < 3:
            print("Warning: Too few valid word pairs for reliable evaluation")
            return None

        correlation, p_value = spearmanr(human_similarities, model_similarities)

        print(f"Word Similarity Evaluation:")
        print(f"Valid pairs: {len(model_similarities)}")
        print(f"Spearman correlation: {correlation:.4f}")
        print(f"P-value: {p_value:.4f}")

        return correlation

    def evaluate_analogies(self, analogy_dataset):
        """
        Evaluate model on word analogy tasks

        Args:
            analogy_dataset: List of tuples (word_a, word_b, word_c, word_d)
                           representing "word_a is to word_b as word_c is to word_d"

        Returns:
            Accuracy on analogy task
        """

        correct = 0
        total = 0
        #('alice', 'girl', 'rabbit', 'animal'),
        for word_a, word_b, word_c, expected_d in analogy_dataset:
            try:
                # Predict word_d
                result = self.wv.most_similar(
                    positive=[word_b, word_c],
                    negative=[word_a],
                    topn=1
                )

                predicted_d = result

                if predicted_d[0][0].lower() == expected_d.lower():
                    correct += 1

                total += 1

            except (KeyError, IndexError):
                # Skip if words not in vocabulary
                continue

        if total == 0:
            print("Warning: No valid analogies found")
            return 0

        accuracy = correct / total

        print(f"Analogy Evaluation:")
        print(f"Valid analogies: {total}")
        print(f"Correct predictions: {correct}")
        print(f"Accuracy: {accuracy:.4f}")

        return accuracy

    def evaluate_odd_one_out(self, word_groups):
        """
        Evaluate model's ability to identify odd words in groups

        Args:
            word_groups: List of lists, each containing words where one doesn't belong

        Returns:
            Accuracy on odd-one-out task
        """

        correct = 0
        total = 0

        for group in word_groups:
            if len(group) < 3:
                continue

            try:
                # Find the word that doesn't match others
                odd_word = self.wv.doesnt_match(group)

                # This is tricky - we need ground truth to evaluate properly
                # For now, just check if the model can identify AN odd word
                correct += 1  # Placeholder - you'd need labeled data
                total += 1

            except KeyError:
                continue

        if total == 0:
            return 0

        accuracy = correct / total

        print(f"Odd-One-Out Evaluation:")
        print(f"  Valid groups: {total}")
        print(f"  Accuracy: {accuracy:.4f}")

        return accuracy

    def analyze_vocabulary_coverage(self, test_texts):
        """
        Analyze how well model vocabulary covers test texts

        Args:
            test_texts: List of text strings

        Returns:
            Coverage statistics
        """

        vocab = set(self.wv.index_to_key)

        total_words = 0
        covered_words = 0
        unknown_words = set()

        for text in test_texts:
            words = text.lower().split()
            total_words += len(words)

            for word in words:
                if word in vocab:
                    covered_words += 1
                else:
                    unknown_words.add(word)

        coverage_ratio = covered_words / total_words if total_words > 0 else 0

        print(f"Vocabulary Coverage Analysis:")
        print(f"  Total words in test: {total_words}")
        print(f"  Covered words: {covered_words}")
        print(f"  Coverage ratio: {coverage_ratio:.4f}")
        print(f"  Unknown words: {len(unknown_words)}")

        return {
            'coverage_ratio': coverage_ratio,
            'unknown_words': list(unknown_words)[:20],  # Show first 20
            'total_unknown': len(unknown_words)
        }

    def compare_with_baseline(self, baseline_model, test_words):
        """
        Compare model performance with baseline model

        Args:
            baseline_model: Another Word2Vec model to compare against
            test_words: List of words to test

        Returns:
            Comparison statistics
        """

        common_words = []
        for word in test_words:
            if word in self.wv and word in baseline_model.wv:
                common_words.append(word)

        if len(common_words) < 3:
            print("Warning: Too few common words for reliable comparison")
            return None

        # Compare similarity patterns
        similarities = []

        for i, word1 in enumerate(common_words[:10]):  # Test subset
            for word2 in common_words[i+1:11]:  # Avoid too many comparisons

                sim1 = self.wv.similarity(word1, word2)
                sim2 = baseline_model.wv.similarity(word1, word2)

                similarities.append((sim1, sim2))

        model_sims = [s[0] for s in similarities]
        baseline_sims = [s[1] for s in similarities]

        correlation, _ = spearmanr(model_sims, baseline_sims)

        print(f"Model Comparison:")
        print(f"  Common vocabulary: {len(common_words)}")
        print(f"  Similarity correlation: {correlation:.4f}")

        return correlation

Note: you may need to restart the kernel to use updated packages.


- Creates a comprehensive evaluation toolkit that tests how well the model performs across different tasks.

In [95]:
# Example evaluation datasets optimized for Alice in Wonderland
word_similarity_pairs = [
    ('alice', 'girl', 8.5),
    ('rabbit', 'hare', 9.2),
    ('queen', 'king', 8.3),
    ('mad', 'crazy', 7.8),
    ('tea', 'party', 6.1),
    ('big', 'small', 2.1),
]

analogy_examples = [
    ('alice', 'girl', 'rabbit', 'animal'),
    ('queen', 'hearts', 'king', 'hearts'),
    ('big', 'bigger', 'small', 'smaller'),
    ('mad', 'hatter', 'march', 'hare'),
]

# Usage example
evaluator = Word2VecEvaluator(model)
sim_score = evaluator.evaluate_word_similarity(word_similarity_pairs)
analogy_score = evaluator.evaluate_analogies(analogy_examples)

Word Similarity Evaluation:
Valid pairs: 4
Spearman correlation: 0.4000
P-value: 0.6000
Analogy Evaluation:
Valid analogies: 3
Correct predictions: 1
Accuracy: 0.3333


- Creates improved evaluation datasets

In [96]:
word = "alice"
if word in model.wv:
    similar_words = model.wv.most_similar(word, topn=10)
    print(f"Most similar words to '{word}':")
    for similar_word, similarity in similar_words:
        print(f"{similar_word}: {similarity:.4f}")
else:
    print("Word is not in the vocabulary.")
    

Most similar words to 'alice':
she: 0.5103
uncomfortable: 0.5008
cautiously: 0.4914
pleaded: 0.4743
thoughtfully: 0.4721
grave: 0.4625
proud: 0.4607
tasted: 0.4560
doubtfully: 0.4531
quickly: 0.4521


 - Tests if the word "alice" exists in your model and shows the 10 most similar words to it.

In [97]:
model.wv.similarity('alice', 'rabbit')

0.28855118

- Calculates the direct similarity score between the words "alice" and "rabbit"